In [240]:
import numpy as np
import pandas as pd
import math
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import sklearn
from sklearn.metrics import precision_score, accuracy_score

In [241]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def amino_encode_table_6(): # key: Amino Acid, value: tensor
    df = pd.read_csv('./6-pc', sep=' ', index_col=0)
    H1 = (df['H1'] - np.mean(df['H1'])) / (np.std(df['H1'], ddof=1))
    V = (df['V'] - np.mean(df['V'])) / (np.std(df['V'], ddof=1))
    P1 = (df['P1'] - np.mean(df['P1'])) / (np.std(df['P1'], ddof=1))
    Pl = (df['Pl'] - np.mean(df['Pl'])) / (np.std(df['Pl'], ddof=1))
    PKa = (df['PKa'] - np.mean(df['PKa'])) / (np.std(df['PKa'], ddof=1))
    NCI = (df['NCI'] - np.mean(df['NCI'])) / (np.std(df['NCI'], ddof=1))
    c = np.array([H1,V,P1,Pl,PKa,NCI], dtype=np.float32)
    amino = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']
    table = {}
    for index,key in enumerate(amino):
        # table[key] = torch.from_numpy(c[0:6, index])
        table[key] = list(c[0:6, index])
    table['X'] = [0,0,0,0,0,0]
    return table

table = amino_encode_table_6()

def padding_seq(original_seq, length=50, pad_value='X'):
    padded_seq = original_seq.ljust(length, pad_value)
    return padded_seq

def seq_to_features(seq):
    features_list = []
    for aa in seq:
        features_list.append(table[aa])
    feature_tensor = torch.Tensor(features_list)
    return feature_tensor

def seq_to_features_ml(seq, conc):
    features_list = []
    for aa in seq:
        t = table[aa].copy()
        t.append(conc)
        features_list += t
    feature_tensor = np.array(features_list, dtype=np.float32)
    return feature_tensor

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

input_size = 6
sequence_length = 50
num_layers = 2

hidden_size = 4
num_epochs = 6000
batch_size = 256
bidirectional = True
learning_rate = 1e-5
early_stop = 1000
weight_decay = 0.00001

seed=10902128
same_seed(seed)

model_path = './class_model.ckpt'

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, bidirectional):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.bi = 1 if bidirectional else 0
        self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, bidirectional=bidirectional, dropout=0.3)
        for name, param in self.rnn.named_parameters():
            if name.startswith('weight'):
                nn.init.xavier_normal_(param)
        self.act = nn.LeakyReLU(0.2)
        self.fc = nn.Linear(hidden_size*(1+self.bi)+1, 1)
        self.sig = nn.Sigmoid()
    
    def forward(self, encoded, conc):
        h0 = torch.zeros(self.num_layers*(1+self.bi), encoded.size(0), self.hidden_size).to(device)
        out, _ = self.rnn(encoded, h0)
        out = out[:, -1, :] # extract last time step
        conc = torch.unsqueeze(conc, 1)
        # out = self.act(out)
        fc_in = torch.cat((out, conc), dim=1)
        ret = self.fc(fc_in)
        r = self.sig(ret)
        return r

class HemolysisDataset(Dataset):
    def __init__(self, parquet):
        df = pd.read_parquet(parquet)
        df['sequence'] = df['sequence'].apply(padding_seq)
        df['encoded'] = df['sequence'].apply(seq_to_features)
        self.features = torch.stack(df['encoded'].values.tolist(), dim=0)
        self.conc = torch.from_numpy(df['concentration'].values)
        self.lysis = torch.from_numpy(df['lysis'].values)
        self.n_samples = df.shape[0]
    
    def __getitem__(self, index):
        return self.features[index], self.conc[index], self.lysis[index]

    def __len__(self):
        return self.n_samples

test_dataset = HemolysisDataset('./test.parquet')
TestLoader = DataLoader(dataset=test_dataset, batch_size=10000000,shuffle=False, num_workers=0)
print(f'number of test data: {len(test_dataset)}')


cuda
number of test data: 277


In [242]:
df = pd.read_parquet('./train.parquet')
Y = df['label'].to_numpy(dtype=int)
X = np.array([]).reshape(0,350)
for i, row in df.iterrows():
    X = np.vstack([X,seq_to_features_ml(padding_seq(row['sequence']), row['concentration'])])

In [243]:
test_df = pd.read_parquet('./test.parquet')
test_Y = test_df['label'].to_numpy(dtype=int)
test_X = np.array([]).reshape(0,350)
for i, row in test_df.iterrows():
    test_X = np.vstack([test_X,seq_to_features_ml(padding_seq(row['sequence']), row['concentration'])])

In [244]:
predictions = np.array([]).reshape(0,len(test_dataset))

In [245]:
# model = RNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, bidirectional=bidirectional).to(device)
# model.load_state_dict(torch.load(model_path))

# model.eval()
# with torch.no_grad():
#     for i, (rnn_input, conc, labels) in enumerate(TestLoader):
#         rnn_input = rnn_input.to(device)
#         conc = conc.to(device)
#         labels = labels.to(device)
#         outputs = model(rnn_input, conc)
#         outputs = outputs.squeeze(1)
#         outputs = outputs.round()
# rnn_pred = outputs.to('cpu').numpy().astype(int)
# predictions = np.vstack([predictions, rnn_pred])

# print(f'RNN acc: {accuracy_score(test_Y, rnn_pred)}, prec: {precision_score(test_Y, rnn_pred)}')

In [246]:
from sklearn import svm
svc_clf = svm.SVC(random_state=seed)
svc_clf.fit(X,Y)
svc_pred = svc_clf.predict(test_X)
predictions = np.vstack([predictions, svc_pred])
print(f'svc acc: {accuracy_score(test_Y, svc_pred)}, prec: {precision_score(test_Y, svc_pred)}')

svc acc: 0.8086642599277978, prec: 0.8102189781021898


In [247]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=seed)
rf_clf.fit(X,Y)
rf_pred = rf_clf.predict(test_X)
predictions = np.vstack([predictions, rf_pred])
print(f'rf acc: {accuracy_score(test_Y, rf_pred)}, prec: {precision_score(test_Y, rf_pred)}')

rf acc: 0.8303249097472925, prec: 0.8321167883211679


In [248]:
# from sklearn.ensemble import AdaBoostClassifier
# ada_clf = AdaBoostClassifier(random_state=seed)
# ada_clf.fit(X,Y)
# ada_pred = ada_clf.predict(test_X)
# predictions = np.vstack([predictions, ada_pred])
# print(f'ada acc: {accuracy_score(test_Y, ada_pred)}, prec: {precision_score(test_Y, ada_pred)}')

In [249]:
# from sklearn.neural_network import MLPClassifier
# mlp_clf = MLPClassifier(max_iter=5000, random_state=seed)
# mlp_clf.fit(X,Y)
# mlp_pred = mlp_clf.predict(test_X)
# predictions = np.vstack([predictions, mlp_pred])
# print(f'mlp acc: {accuracy_score(test_Y, mlp_pred)}, prec: {precision_score(test_Y, mlp_pred)}')

In [250]:
# from sklearn.neighbors import KNeighborsClassifier
# knn_clf = KNeighborsClassifier()
# knn_clf.fit(X,Y)
# knn_pred = knn_clf.predict(test_X)
# predictions = np.vstack([predictions, knn_pred])
# print(f'knn acc: {accuracy_score(test_Y, knn_pred)}, prec: {precision_score(test_Y, knn_pred)}')

In [251]:
import xgboost as xgb
xgb_clf = xgb.XGBClassifier(random_state=seed)
xgb_clf.fit(X,Y)
xgb_pred = xgb_clf.predict(test_X)
predictions = np.vstack([predictions, xgb_pred])
print(f'xgb acc: {accuracy_score(test_Y, xgb_pred)}, prec: {precision_score(test_Y, xgb_pred)}')


xgb acc: 0.8231046931407943, prec: 0.8345864661654135


In [252]:
display(predictions.shape)

(3, 277)

In [253]:
predict = predictions.sum(axis=0)
predict[predict <= (len(predictions)//2)] = 0
predict[predict > (len(predictions)//2)] = 1
predict

array([0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1.,
       1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0.,
       0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0.,
       1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1.,
       0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1.,
       1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
       0., 1., 0., 1., 0.

In [254]:
final_pred = predict
print(f'final acc: {accuracy_score(test_Y, final_pred)}, prec: {precision_score(test_Y, final_pred)}')

final acc: 0.8375451263537906, prec: 0.8444444444444444


In [255]:
# threshold=20: final acc: 0.8375451263537906,  prec: 0.8444444444444444    (All without knn, rnn, mlp, ada)
# threshold=30: final acc: 0.8125,              prec: 0.8125                (All without knn, rnn, mlp, ada)
# threshold=40: final acc: 0.8,                 prec: 0.8288288288288288    (All without knn, rnn, ada)